In [4]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});
MathJax.Hub.Queue(
  ["resetEquationNumbers", MathJax.InputJax.TeX],
  ["PreProcess", MathJax.Hub],a
  ["Reprocess", MathJax.Hub]
);

<IPython.core.display.Javascript object>

# Derivatives of KL-Divergence for various parameterizations of a multivariate Gaussian covariance

> *These notes provide the derivatives of the KL-divergence $D_{\text{KL}}\left[ Q(\mathbf z) \| \Pr(\mathbf z)\right]$ between two multivariate Gaussian distributions $Q(\mathbf z)$ and $\Pr(\mathbf z)$ with respect to various parameterizations of the covariance matrix of $Q$. This is useful for variational Gaussian process inference, where clever parameterizations of the posterior covariance are required to make the problem computationally tractable. Tables for differentiating matrix-valued functions can be found in [The Matrix Cookbook](https://www2.imm.dtu.dk/pubdb/pubs/3274-full.html).*

Consider a Bayesian inference problem with observed data $\mathbf y$, and latent variables $\mathbf z$ to be inferred. Assume that the prior $\Pr(\mathbf z)$ is a multivariate Gaussian, $\mathbf z\sim\mathcal N(\boldsymbol\mu_0,\boldsymbol\Sigma_0)$. We are interested in the posterior distribution of $\mathbf z$ given the observations $\mathbf y$ 

\begin{equation}\begin{aligned}
\Pr(\mathbf z | \mathbf y) = \Pr(\mathbf y | \mathbf z)
\frac{\Pr(\mathbf z)}{\Pr(\mathbf y)}
\end{aligned}\end{equation}

Unless $\Pr(\mathbf y | \mathbf z)$ is also a multivariate Gaussian, there will be no simple closed-form for the posterior $\Pr(\mathbf z | \mathbf y)$. The variational Bayesian approach approximates this posterior $\Pr(\mathbf z | \mathbf y)$ with a simpler more tractable distribution. Here, we consider the case of multivariate Gaussian distribution:

\begin{equation}\begin{aligned}
\Pr(\mathbf z | \mathbf y)
\approx Q(\mathbf z) = \mathcal N(\boldsymbol\mu_q,\boldsymbol\Sigma_q)
\end{aligned}\end{equation}

The parameters $\Theta = (\boldsymbol\mu_q,\boldsymbol\Sigma_q)$ of $Q(\mathbf z)$ are fit buy minimizing the [Kullback-Leibler (KL) divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) from the true posterior $\Pr(\mathbf z | \mathbf y)$ to $Q(\mathbf z)$:

\begin{equation}\begin{aligned}
\Theta\gets\underset{\Theta}{\operatorname{argmin}}
D_{\text{KL}}\left[ Q(\mathbf z) \| \Pr(\mathbf z | \mathbf y)\right]
\end{aligned}\end{equation}

This is equivalent to minimizing the KL divergenece $D_{\text{KL}}\left[ Q(\mathbf z) \| \Pr(\mathbf z)\right]$ from the prior to the posterior, while maximizing the expected log-likelihood $\left<\Pr(\mathbf y | \mathbf z)\right>_Q$:

\begin{equation}\begin{aligned}
\boldsymbol\mu_q,\boldsymbol\Sigma_q
&\gets
\underset{\boldsymbol\mu_q,\boldsymbol\Sigma_q}{\operatorname{argmin}}
\mathcal L(\boldsymbol\mu_q,\boldsymbol\Sigma_q)
\\
\mathcal L(\boldsymbol\mu_q,\boldsymbol\Sigma_q)
&=
D_{\text{KL}}\left[ Q(\mathbf z) \| \Pr(\mathbf z | \mathbf y)\right]
\\
&=
\left<
\ln\frac{Q(\mathbf z)}
{\Pr(\mathbf z | \mathbf y)}
\right>_{Q}
\\
&=
\left<
\ln Q(\mathbf z)
\right>_Q
-
\left<
\ln\Pr(\mathbf z | \mathbf y)
\right>_Q
\\
&=
\left<
\ln Q(\mathbf z)
\right>_Q
-
\left<
\ln\left(
\Pr(\mathbf y | \mathbf z)
\frac
{\Pr(\mathbf z)}
{\Pr(\mathbf y)}
\right)
\right>_Q
\\
&=
\left<\ln \frac{Q(\mathbf z)}{\Pr(\mathbf z)}\right>_Q-\left<\ln\Pr(\mathbf y | \mathbf z)
\right>_Q
+\left<\ln\Pr(\mathbf y)\right>_Q\\
&=
D_{\text{KL}}\left[ Q(\mathbf z) \| \Pr(\mathbf z)\right]
-\left<\ln\Pr(\mathbf y | \mathbf z)\right>_Q+c,
\end{aligned}\end{equation}

where $\langle\cdot\rangle_Q$ denotes averaging with respect to $Q(\mathbf z)$, and the probability of the observations $\Pr(\mathbf y)$ can be neglected, since it is a constant ("$c$") that does not depend on $Q$. Since both $Q(\mathbf z)$ and $\Pr(\mathbf z)$ are multivariate Gaussian, the KL divergence $D_{\text{KL}}\left[ Q(\mathbf z) \| \Pr(\mathbf z)\right]$ [has the closed form](https://en.wikipedia.org/wiki/Multivariate_normal_distribution#Kullback%E2%80%93Leibler_divergence)

\begin{equation}\begin{aligned}
D_{\text{KL}}\left[ Q(\mathbf z) \| \Pr(\mathbf z)\right]
&=
\frac 1 2 \left\{
(\boldsymbol\mu_0-\boldsymbol\mu_q)^\top 
\boldsymbol\Sigma_0^{-1}
(\boldsymbol\mu_0-\boldsymbol\mu_q)
+
\operatorname{tr}\left(
\boldsymbol\Sigma_0^{-1}
\boldsymbol\Sigma_q
\right)
+
\ln|
\boldsymbol\Sigma_0^{-1}
\boldsymbol\Sigma_q
|
\right\}
+\text{constant.}
\end{aligned}
\label{dkl}
\end{equation}

In these notes, we consider the first and second derivatives of $\mathcal L$ with respect to the posterior parameters $(\boldsymbol\mu_q,\boldsymbol\Sigma_q)$. Gradients of the Gaussian LK-divergence $D_{\text{KL}}\left[ Q(\mathbf z) \| \Pr(\mathbf z)\right]$ are solved explicitly. Gradients of the expected negative log-likelihood, $\mathcal \ell = -\left<\ln\Pr(\mathbf y)\right>_Q$, depend on the specific choice for the likelihood and will be addressed elsewheres. The derivatives of $\mathcal L$ in $\boldsymbol\mu_q$ are straightforward:


\begin{equation}\begin{aligned}
\partial_{\boldsymbol\mu_q}\mathcal L
&=
\partial_{\boldsymbol\mu_q}\ell
+
\boldsymbol\Sigma_0^{-1}
(\boldsymbol\mu_q-\boldsymbol\mu_z)
\\
\operatorname H_{\boldsymbol\mu_q}\mathcal L
&=
\operatorname H_{\boldsymbol\mu_q}\ell
+
\boldsymbol\Sigma_0^{-1}.
\end{aligned}\end{equation}

The remainder of the document considers derivatives in $\boldsymbol\Sigma_q$.

# Derivatives for various parameterizations of ${\boldsymbol\Sigma}_q$

We evaluate the following approximations for the posterior covariance: 
1. Optimizing the full $\boldsymbol\Sigma_q$ directly. This is generally inadvisable, but is provided for completeness and comparison to the other approaches. 
3. A low-rank approximation $\boldsymbol\Sigma_q\approx\mathbf X\mathbf X^\top$, where $\mathbf X$ is a tall, thin matrix. This captures the principle subspace of $\boldsymbol\Sigma_q$.
1. A diagonal approximation $\boldsymbol\Sigma_q\approx\mathbf A^\top \operatorname{diag}[\mathbf v] \mathbf A$, where $\mathbf A$ is a fixed spatial convolution.
2. A modification of (3) where $\mathbf A$ is a local Gaussian blur.
2. An inverse diagonal approximation $\boldsymbol\Sigma_q\approx[\boldsymbol\Sigma_0^{-1} + \operatorname{diag}[\mathbf p]]^{-1}$, where $\boldsymbol\Sigma_0$ is the prior covariance and $\mathbf p$ is a vector of the inverse variance (precision) at each spatial location.
4. A reduced Fourier-space representation $\mathbf F^\top \mathbf Q \mathbf Q^\top \mathbf F$, where $\mathbf F$ is the unitary 2D Fourier transform, with frequencies that are zero in the prior $\boldsymbol\Sigma_0$ discarded, and $\mathbf Q$ is a lower-triangular matrix. Since the posterior cannot assign probability mass where the prior $\boldsymbol\Sigma_0$ is zero, this fully parameterizes $\boldsymbol\Sigma_q$ in a compact way. The Fourier transform can be computed quickly using the Fast Fourier Transform (FFT).

## Fully-parameterizing $\boldsymbol\Sigma_q$

It is unusual to optimize the covarince of the variational posterior, $\boldsymbol\Sigma_q$, directly. For one, this matrix must be constrained to be positive definite, and it is easier to ensure this via e.g. parameterising in terms of a triangular factor $\mathbf X$ such that $\boldsymbol\Sigma_q = \mathbf X\mathbf X^\top$. Additionally, in most Gaussian-process inference problems, $\boldsymbol\Sigma_q$ will be sufficiently large that optimizing the full-rank representation is computatoinally prohibative. 

Neverheless, we present the derivatives in $\boldsymbol\Sigma_q$ here. It will be useful to compare the derivatives for various parameterizations of $\boldsymbol\Sigma_q$ to these.

### Gradient

The gradient and Hessian of $\mathcal L$ with respect to $\boldsymbol\Sigma_q$ are more complicated. An analytic derivative in $\boldsymbol\Sigma_q$ can be obtained using the various identities provided in [The Matrix Cookbook](https://www2.imm.dtu.dk/pubdb/pubs/3274-full.html).

\begin{equation}\begin{aligned}
\partial_{\boldsymbol\Sigma_q}\mathcal L
=
\partial_{\boldsymbol\Sigma_q}\ell
+
\frac 1 2 \left(
\boldsymbol\Sigma_z^{-1}
+
\boldsymbol\Sigma_q^{-\top}
\right)
\end{aligned}\end{equation}

### Hessian-vector-product 

The Hessian in $\boldsymbol\Sigma_q$ is a fourth-order tensor. It's simpler to express the Hessian in terms of a Hessian-vector product, which can be used with [Krylov subspace](https://en.wikipedia.org/wiki/Krylov_subspace) solvers to efficiently compute the update in Newton's method. These solvers are implemented, for example, in Matlab and Scipy's "minres" function. Considering an $N\times N$ matrix $\mathbf M$, the Hessian-vector product is given by 

\begin{equation}\begin{aligned}
\partial_{\boldsymbol\Sigma_q}
\left< \partial_{\boldsymbol\Sigma_q}\mathcal L,\mathbf M\right>
=
\partial_{\boldsymbol\Sigma_q}
\operatorname{tr}\left[
(\partial_{\boldsymbol\Sigma_q}\mathcal L)^\top \mathbf M\right]
=
\partial_{\boldsymbol\Sigma_q}\left< \partial_{\boldsymbol\Sigma_q}\ell,\mathbf M\right>
+
\operatorname{tr}\left[\frac 1 2 \left(\boldsymbol\Sigma_0^{-1}+\boldsymbol\Sigma_q^{-\top}\right)^\top\mathbf M\right]
\end{aligned}\end{equation}

where $\langle\cdot,\cdot\rangle$ denotes the scalar product. The Hessian-vector product for terms arising from the KL divergence is straightforward:

\begin{equation}\begin{aligned}
\partial_{\boldsymbol\Sigma_q}
\operatorname{tr}\left[
\frac 1 2 \left(
\boldsymbol\Sigma_0^{-1}
+
\boldsymbol\Sigma_q^{-\top}
\right)
^\top
\mathbf M
\right]
&=
\frac 1 2 
\partial_{\boldsymbol\Sigma_q}
\operatorname{tr}\left[
\boldsymbol\Sigma_q^{-1}
\mathbf M
\right]
=
-\frac 1 2 
\boldsymbol\Sigma_q^{-1}
\mathbf M^\top
\boldsymbol\Sigma_q^{-1}
\end{aligned}\end{equation}

### ${\boldsymbol\Sigma}_q\approx{\mathbf X}{{\mathbf X}^{\top}}$

We consider an approximate posterior covariance of the form

\begin{equation}\begin{aligned}
\boldsymbol\Sigma^{-1} &\approx \mathbf X \mathbf X^\top,\,\,\,\,\mathbf X\in\mathbb R^{L^2\times K}
\end{aligned}\end{equation}

where $\mathbf X$ is a tall, thin matrix, with as many rows as there are spatial bins ($L^2$), and $K<L^2$ columns.

If $\mathbf X$ is not full rank, the log-determinant $\ln|{\boldsymbol\Sigma}_q|=\ln|\mathbf X\mathbf X^\top|$ in $\eqref{dkl}$ will diverge to $-\infty$, due to the zero eigenvalues in the null space of $\mathbf X$. However, since this null-space is not being optimized, it does not affect our gradient. It is sufficient to replace the log-determinant with that of the reduced-rank representation, $\ln|\mathbf X^\top\mathbf X|$.

For the graident, the Matrix Cookbook provides $\partial_{\mathbf X}\ln|\mathbf X^\top\mathbf X| = {\mathbf X^{+}}^\top$


\begin{equation}\begin{aligned}
\partial_{\mathbf X}
\mathcal L &=
(\partial_{\boldsymbol\Sigma_q}\ell) \mathbf X
+
\boldsymbol\Sigma_0^{-1}
\mathbf X - {\mathbf X^{+}}^\top
\end{aligned}\end{equation}


\begin{equation}\begin{aligned}
\partial_{\mathbf X} \left< \partial_{\boldsymbol\Sigma_q} \mathcal L , \mathbf M \right>
&=
\partial_{\mathbf X} \left< 
\left(
\partial_{\boldsymbol\Sigma_q}\ell
+ \boldsymbol\Sigma_0^{-1}\right)\mathbf X - {\mathbf X^{+}}^\top
, \mathbf M \right>
\\
&=
\partial_{\mathbf X}
\operatorname{tr}\left[
\mathbf X^\top
\left(
\partial_{\boldsymbol\Sigma_q}\ell
\right)
\mathbf M
+ 
\mathbf X^\top \boldsymbol\Sigma_0^{-1}\mathbf M
- 
{\mathbf X^{+}}\mathbf M
\right]
\\
&=
\left(\partial_{\boldsymbol\Sigma_q}\ell\right)
\mathbf M
+ 
\boldsymbol\Sigma_0^{-1}\mathbf M
+{\mathbf X^+}^\top \mathbf M^\top {\mathbf X^+}^\top - (\mathbf I - {\mathbf X^+}^\top \mathbf X^\top) \mathbf M \mathbf X^+ {\mathbf X^+}^\top
\end{aligned}\end{equation}

 (These same derivaties also work to optimize the full rank ${\boldsymbol\Sigma}_q$ if $\mathbf X$ is a triangular (Choleskey) factorization of ${\boldsymbol\Sigma}_q$)

The gradient and Hessian of this loss function in $\mathbf v$ are : 

\begin{equation}\begin{aligned}
%%%% GRADIENT IN v
\operatorname{\nabla}_{\mathbf v}
\mathcal L &=\tfrac 1 2 \left\{
\mathbf G^\top (\mathbf n \circ \langle\boldsymbol\lambda\rangle)
+ \operatorname{diag}[\mathbf A \boldsymbol\Sigma_0^{-1} \mathbf A^\top]
- \tfrac 1 {\mathbf v}
\right\}
\\
%%%% HESSIAN IN v
\operatorname{H}_{\mathbf v}
\mathcal L &=\tfrac 1 2 \left\{
\tfrac 1 2 
\mathbf G \operatorname{diag}[\mathbf n \circ \langle\boldsymbol\lambda\rangle] \, \mathbf G 
+ \operatorname{diag}\left[\tfrac 1 {\mathbf v^2}\right]
\right\}
\end{aligned}\end{equation}

This follows from the usual matrix and scalar derivatives, with the exception of the term $\mathbf n^\top \langle\boldsymbol\lambda\rangle$. These can be obtained by considering the derivative with respect to single elements of $\mathbf v$ (note: $\mathbf G$ is symmetric): 

\begin{equation}\begin{aligned}
\tfrac{d}{dv_j} \mathbf n^\top\langle\boldsymbol\lambda\rangle 
&=  \mathbf n^\top
\tfrac{d}{dv_j}  \langle\boldsymbol\lambda\rangle 
\\
&=  \mathbf n^\top
\tfrac{d}{dv_j} \exp\left(\boldsymbol\mu + \tfrac12\mathbf G\mathbf v\right)
\\
&= \tfrac{d}{dv_j} \sum_k
n_k \exp\left[\mu_k + \tfrac12(\mathbf G\mathbf v)_k\right]
\\
&= \sum_k
n_k \left[
\langle\lambda_k\rangle
\cdot \tfrac12 \tfrac{d}{dv_j} (\mathbf G\mathbf v)_k
\right]
\\
&= 
\tfrac12 \sum_k
 n_k 
\langle\boldsymbol\lambda_k\rangle
\textstyle \mathbf G_{kj} 
\\
&= 
\tfrac12 \left[ \mathbf G^\top(\mathbf n\circ\langle\boldsymbol\lambda\rangle) \right]_j
\\\\
\tfrac{d^2}{dv_iv_j} \mathbf n^\top\langle\boldsymbol\lambda\rangle 
&= 
\tfrac12 \left[ \sum_k n_k \tfrac{d}{dv_i} \langle\boldsymbol\lambda_k\rangle\textstyle \mathbf G_{kj} \right]_j
\\
&= 
\tfrac14 \left[ \sum_k n_k 
\langle\boldsymbol\lambda_k\rangle
\textstyle \mathbf G_{jk} 
\textstyle \mathbf G_{ki} 
\right]_j
\\
&= 
\tfrac 1 4 \left[\mathbf G \operatorname{diag}[\mathbf n\circ \langle\boldsymbol\lambda\rangle] \, 
\mathbf G
\right]_{ij}
\end{aligned}\end{equation}